# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: We loaded the Azure Search Engine with enriched PDFs in index: "cogsrch-index-files"
- **Notebook 02**: We loaded more information to the Search Engine this time using a CSV file with 90k rows/articles in index: "cogsrch-index-csv"
- **Notebook 03**: We added AzureOpenAI GPT models to enhance the the production of the answer by using Utility Chains of LLMs
- **Notebook 04**: We manually loaded an index with large/complex PDFs information , "cogsrch-index-books-vector"
- **Notebook 05**: We added memory to our system in order to power a conversational Chat Bot
- **Notebook 06**: We introduced Agents and Tools and built the first Skill/Agent, that can do RAG over a search engine
- **Notebook 07**: We build a second Agent (Pandas) in order to be able to solve a more complex task: ask questions to Tabular datasets
- **Notebook 08**: We used a SQL Agent in order to talk to a SQL Database directly
- **Notebook 09**: We used another  Agent in order to talk to the Bing Search API and create a Bing Chat Clone and implemented callbacks for real-time streaming and tool information
- **Notebook 10**: We built an API Agent that can translate a question into the right API calls, giving us the capability to talk to any datasource that provides a RESTFul API.


We are missing one more thing: **How do we glue all these features together into a very smart GPT Smart Search Engine Chat Bot?**

We want a virtual assistant for our company that can get the question, think what tool to use, then get the answer. The goal is that, regardless of the source of the information (Search Engine, Bing Search, SQL Database, CSV File, JSON File, APIs, etc), the Assistant can answer the question correctly using the right tool.

In this Notebook we are going to create that "brain" Agent (also called Master Agent), that:

1) understands the question, interacts with the user 
2) talks to other specialized Agents that are connected to diferent sources
3) once it get's the answer it delivers it to the user or let the specialized Agent to deliver it directly

This is the same concept of [AutoGen](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/): Agents talking to each other.

![image](https://www.microsoft.com/en-us/research/uploads/prod/2023/09/AutoGen_Fig1.png)

In [1]:
import os
import random
import json
import requests
from langchain_openai import AzureChatOpenAI
from langchain.agents import ,AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables import ConfigurableField

#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent, 
    SQLSearchAgent, 
    ChatGPTTool, 
    BingSearchAgent, 
    APISearchAgent, 
    reduce_openapi_spec
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

MODEL_DEPLOYMENT_NAME = "gpt-35-turbo-1106" # Reminder: gpt-35-turbo models will create parsing errors and won't follow instructions correctly 

In [2]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Get the Tools - DocSearch Agent, CSV Agent, SQL Agent, Web Search Agent, ChatGPT, API Agent

**Consider the following concept:** Agents, which are essentially software entities designed to perform specific tasks, can be equipped with tools. These tools themselves can be other agents, each possessing their own set of tools. This creates a layered structure where tools can range from code sequences to human actions, forming interconnected chains. Ultimately, you're constructing a network of agents and their respective tools, all collaboratively working towards solving a specific task (This is what ChatGPT is). This network operates by leveraging the unique capabilities of each agent and tool, creating a dynamic and efficient system for task resolution.

In the file `common/utils.py` we created Agent Tools Classes for each of the Functionalities that we developed in prior Notebooks. 

In [33]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager).configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt35",
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager),
)

# Uncomment the below line if you want to see the responses being streamed/typed
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)

In [4]:
# DocSearchAgent is our Custom Tool Class (Agent) created for Azure Cognitive Search + OpenAI searches
text_indexes = ["cogsrch-index-files", "cogsrch-index-csv", "cogsrch-index-books"]
doc_search = DocSearchAgent(llm=llm, indexes=text_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           callback_manager=cb_manager, verbose=False)

In [5]:
# BingSearchAgent is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchAgent(llm=llm, k=5, callback_manager=cb_manager)

In [6]:
## CSVTabularAgent is a custom Tool class crated to Q&A over CSV files
file_url = "./data/all-states-history.csv"
csv_search = CSVTabularAgent(path=file_url, llm=llm, callback_manager=cb_manager)

In [7]:
## SQLDbAgent is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLSearchAgent(llm=llm, k=30, callback_manager=cb_manager)

In [8]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager)

In [9]:
## APISearchAgent is a custom Tool class created to talk to any API 
# The path to your JSON file
file_path = './data/openapi.json'

# Open the file for reading
with open(file_path, 'r') as file:
    # Parse the JSON data from the file
    spec = json.load(file)

api_search = APISearchAgent(llm=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            llm_search=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            api_spec=str(reduce_openapi_spec(spec)),
                            callback_manager=cb_manager)

### Variables/knobs to use for customization

As you have seen so far, there are many knobs that you can dial up or down in order to change the behavior of your GPT Smart Search engine application, these are the variables you can tune:

- <u>llm</u>:
  - **deployment_name**: this is the deployment name of your Azure OpenAI model. This of course dictates the level of reasoning and the amount of tokens available for the conversation. For a production system you will need gpt-4-32k. This is the model that will give you enough reasoning power to work with agents, and enough tokens to work with detailed answers and conversation memory.
  - **temperature**: How creative you want your responses to be
  - **max_tokens**: How long you want your responses to be. It is recommended a minimum of 500
- <u>Tools</u>: To each tool you can add the following parameters to modify the defaults (set in utils.py), these are very important since they are part of the system prompt and determines what tool to use and when.
  - **name**: the name of the tool
  - **description**: when the brain agent should use this tool
- <u>DocSearchAgent</u>: 
  - **k**: The top k results per index from the text search action
  - **similarity_k**: top k results combined from the vector search action
  - **reranker_th**: threshold of the semantic search reranker. Picks results that are above the threshold. Max possible score=4
- <u>BingSearchAgent</u>:
  - **k**: The top k results from the bing search action
- <u>SQLSearchAgent</u>:
  - **k**: The top k results from the SQL search action. Adds TOP clause to the query
  
in `utils.py` you can also tune:
- <u>model_tokens_limit</u>: In this function you can edit what is the maximum allows of tokens reserve for the content. Remember that the remaining will be for the system prompt plus the answer

### Test the Tools

In [10]:
# Test the Documents Search Tool with a question we know it doesn't have the knowledge for
printmd(doc_search.run("what is the weather today in Dallas?"))

Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'weather in Dallas today'}`



I'm sorry, but I couldn't find the specific weather information for Dallas today. If you'd like, I can help you find a reliable source for up-to-date weather forecasts for Dallas.

I'm sorry, but I couldn't find the specific weather information for Dallas today. If you'd like, I can help you find a reliable source for up-to-date weather forecasts for Dallas.

In [11]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(doc_search.run("How Covid affects obese people? and elderly?"))

Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'How does Covid affect obese people?'}`



Agent Action: 
Invoking: `docsearch` with `{'query': 'How does Covid affect elderly people?'}`



### How Covid Affects Obese People

Obese individuals are at a higher risk of developing severe illness from COVID-19. Here are some key findings related to the impact of COVID-19 on obese individuals:

1. **Increased Severity in Critically Ill Patients**: Obesity is highly frequent among critically ill patients with SARS-Cov-2 infection, and it remains challenging to elucidate the mechanisms by which the severity of SARS-Cov-2 is increased in the context of obesity<sup><a href="https://doi.org/10.1002/oby.22867; https://www.ncbi.nlm.nih.gov/pubmed/32365275/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[1]</a></sup>.

2. **Cardiac Complications**: Hospitalized patients with COVID-1

### How Covid Affects Obese People

Obese individuals are at a higher risk of developing severe illness from COVID-19. Here are some key findings related to the impact of COVID-19 on obese individuals:

1. **Increased Severity in Critically Ill Patients**: Obesity is highly frequent among critically ill patients with SARS-Cov-2 infection, and it remains challenging to elucidate the mechanisms by which the severity of SARS-Cov-2 is increased in the context of obesity<sup><a href="https://doi.org/10.1002/oby.22867; https://www.ncbi.nlm.nih.gov/pubmed/32365275/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[1]</a></sup>.

2. **Cardiac Complications**: Hospitalized patients with COVID-19, particularly those who are overweight or obese, presented cardiac complications including myocarditis, arrhythmias, heart failure (HF), and sudden death<sup><a href="https://doi.org/10.1002/oby.22867; https://www.ncbi.nlm.nih.gov/pubmed/32365275/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[1]</a></sup>.

3. **Risk of Severe COVID-19**: Obese patients had increased odds of progressing to severe COVID-19, with obese men having even higher odds of developing severe disease compared to those with normal weight<sup><a href="https://doi.org/10.2337/dc20-0576; https://www.ncbi.nlm.nih.gov/pubmed/32409502/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[2]</a></sup>.

4. **Impact on Severity**: Obese patients tended to have symptoms of cough and fever compared with non-obese patients, and they had increased odds of developing severe COVID-19<sup><a href="https://doi.org/10.2337/dc20-0576; https://www.ncbi.nlm.nih.gov/pubmed/32409502/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[2]</a></sup>.

5. **Global Impact**: The WHO considers obesity a major risk factor for becoming seriously ill with COVID-19, and a significant proportion of patients in critical care units are either overweight or obese<sup><a href="https://doi.org/10.1002/oby.22844; https://www.ncbi.nlm.nih.gov/pubmed/32314871/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[3]</a></sup>.

### How Covid Affects Elderly People

Elderly individuals are at a higher risk of more serious and fatal illness associated with COVID-19. Here are some key findings related to the impact of COVID-19 on elderly individuals:

1. **Increased Mortality Risk**: Mortality data indicates a risk of mortality of 3.6% for people in their 60s, which increases to 8.0% and 14.8% for people in their 70s and over 80s<sup><a href="https://doi.org/10.1111/jocn.15274; https://www.ncbi.nlm.nih.gov/pubmed/32239784/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[4]</a></sup>.

2. **Higher Incidence and Severity**: The current SARS-CoV-2 pandemic is affecting elderly people worldwide with greater incidence and severity, with a significant proportion of coronavirus hospitalizations corresponding to those over 60 years of age<sup><a href="https://doi.org/10.1016/j.enfcli.2020.05.004; https://www.ncbi.nlm.nih.gov/pubmed/32425485/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[5]</a></sup>.

3. **Risk of Infection Across Age Groups**: Protecting elderly people from coronavirus infection could reduce the risk of infection among themselves and ameliorate the risks of virus infection among other age groups<sup><a href="http://medrxiv.org/cgi/content/short/2020.05.17.20105049v1?rss=1?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[6]</a></sup>.

4. **Complications and Severity**: Complications of COVID-19 have been particularly severe among older adults, and public policy goals should prioritize pandemic preparedness in nursing homes and support programs for community-dwelling older adults<sup><a href="https://doi.org/10.1172/jci.insight.139292; https://www.ncbi.nlm.nih.gov/pubmed/32302293/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">[7]</a></sup>.

These findings underscore the increased vulnerability of obese and elderly individuals to severe illness and complications from COVID-19.

---
The references for the above information can be found by clicking on the footnote numbers.

In [14]:
# Test the Bing Search Agent
printmd(www_search.run("Who are the family member names of the current president of India?"))

Tool: bing
Agent Action: 
Invoking: `Searcher` with `family members of the current president of India`



Agent Action: 
Invoking: `Searcher` with `current president of India`



The current President of India is Droupadi Murmu, who took office on July 25, 2022. She is the 15th and current President of India, the first person from the tribal community, and the second woman after Pratibha Patil to hold the office.

As for the family members of President Droupadi Murmu, I was unable to find specific information about her family members in the search results. If you have any other questions or need further information, feel free to ask!

The current President of India is Droupadi Murmu, who took office on July 25, 2022. She is the 15th and current President of India, the first person from the tribal community, and the second woman after Pratibha Patil to hold the office.

As for the family members of President Droupadi Murmu, I was unable to find specific information about her family members in the search results. If you have any other questions or need further information, feel free to ask!

In [15]:
# Test the CSV Agent
printmd(csv_search.run("how many rows does the file have?"))

Tool: csvfile
Agent Action: 
Invoking: `python_repl_ast` with `{'query': 'df.shape'}`



The file has 20,780 rows.

Explanation:
I used the `df.shape` attribute to get the number of rows in the dataframe. The first value in the output represents the number of rows, which is 20,780.

The file has 20,780 rows.

Explanation:
I used the `df.shape` attribute to get the number of rows in the dataframe. The first value in the output represents the number of rows, which is 20,780.

In [16]:
# Test the SQL Search Agent
printmd(sql_search.run("How many people in total died california in each state of the west coast in July 2020?"))

Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with ``



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`



Agent Action: 
Invoking: `sql_db_query` with `SELECT state, SUM(death) as total_deaths
FROM covidtracking
WHERE state IN ('CA', 'OR', 'WA') AND date LIKE '2020-07%'
GROUP BY state`



The total number of deaths in California, Oregon, and Washington in July 2020 are as follows:
- California: 229,362 deaths
- Oregon: 7,745 deaths
- Washington: 44,440 deaths

Here's the SQL query I used to obtain the results:
```sql
SELECT state, SUM(death) as total_deaths
FROM covidtracking
WHERE state IN ('CA', 'OR', 'WA') AND date LIKE '2020-07%'
GROUP BY state
``` 

Explanation:
1. I used the `covidtracking` table to retrieve the total number of deaths in California (CA), Oregon (OR), and Washington (WA) in July 2020.
2. I filtered the data based on the state and date using the `WHERE` clause.
3. I used the `SUM` function to calculate the total de

The total number of deaths in California, Oregon, and Washington in July 2020 are as follows:
- California: 229,362 deaths
- Oregon: 7,745 deaths
- Washington: 44,440 deaths

Here's the SQL query I used to obtain the results:
```sql
SELECT state, SUM(death) as total_deaths
FROM covidtracking
WHERE state IN ('CA', 'OR', 'WA') AND date LIKE '2020-07%'
GROUP BY state
``` 

Explanation:
1. I used the `covidtracking` table to retrieve the total number of deaths in California (CA), Oregon (OR), and Washington (WA) in July 2020.
2. I filtered the data based on the state and date using the `WHERE` clause.
3. I used the `SUM` function to calculate the total deaths for each state and grouped the results by state using the `GROUP BY` clause.
4. The query returned the total deaths for each state.

In [10]:
# Test the ChatGPTWrapper Search Tool
printmd(chatgpt_search.run("what is the function in python that allows me to get a random number?"))

Tool: chatgpt


In Python, you can use the `random` module to generate random numbers. The `random` module provides several functions for generating random numbers, such as `random()`, `randint()`, `choice()`, and `shuffle()`. Here's an example of how to use the `randint()` function to generate a random integer between a specified range:

```python
import random
random_number = random.randint(1, 10)  # Generates a random integer between 1 and 10
print(random_number)
```

This will output a random integer between 1 and 10. You can replace the arguments of `randint()` with your desired range.

### Define what tools are we going to give to our brain agent

Go to `common/utils.py` to check the tools definition and the instructions on what tool to use when

In [12]:
tools = [www_search, sql_search, doc_search, chatgpt_search]

**Note**: Notice that since both the CSV file and the SQL Database have the same exact data, we are only going to use the SQLDBTool since it is faster and more reliable

### Initialize the brain agent

In [16]:
agent = create_openai_tools_agent(llm.with_config(configurable={"model": "gpt35"}), tools, CUSTOM_CHATBOT_PROMPT)

In [17]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False, callback_manager=cb_manager)

In [18]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [19]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [20]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

### Let's talk to our GPT Smart Search Engine chat bot now

In [21]:
# This question should not use any tool, the brain agent should answer it without the use of any tool
printmd(brain_agent_executor.invoke({"question": "Hi, I'm Pablo Marin, how are you doing today?"}, config=config)["output"])

I'm here and ready to assist you, Pablo. How can I help you today?

I'm here and ready to assist you, Pablo. How can I help you today?

In [23]:
# This question should not use any tool either
printmd(brain_agent_executor.invoke({"question": "What is your name?"}, config=config)["output"])

I'm an AI assistant here to help you with any questions or tasks you have. How can I assist you today?

I'm an AI assistant here to help you with any questions or tasks you have. How can I assist you today?

In [24]:
printmd(brain_agent_executor.invoke({"question": "bing, I need to take my girlfriend to dinner tonight in downtown Chicago. Please give me options for Italian and Sushi as well"}, 
                                    config=config)["output"])

Agent Action: 
Invoking: `bing` with `Italian restaurants in downtown Chicago`



Tool: bing
Agent Action: 
Invoking: `Searcher` with `{'query': 'Italian restaurants in downtown Chicago'}`



Based on the search results, here are some Italian restaurants in downtown Chicago:

1. **Madison Street Location** - This restaurant embodies historic Chicago dining, featuring timeless songs by Sinatra and Martin. It offers a classic dining experience with deep, rich wooden walls adorned with numerous photos of famous athletes and entertainers. [More Info](https://www.opentable.com/cuisine/best-italian-restaurants-downtown-chicago-il)

2. **Giordano's** - Known for serving authentic Chicago-style deep dish pizza with quality toppings, this restaurant also offers a selection of sandwiches, pastas, and buffalo wings in a clean, bright space. [More Info](https://www.tripadvisor.com/Restaurants-g35805-c26-zfn7778523-Chicago_Illinois.html)

3. **Monteverde** - Located in the West Loop, this Italian r

Based on the search results, here are some options for Italian and Sushi restaurants in downtown Chicago:

### Italian Restaurants
1. **Madison Street Location** - This restaurant embodies historic Chicago dining, featuring timeless songs by Sinatra and Martin. It offers a classic dining experience with deep, rich wooden walls adorned with numerous photos of famous athletes and entertainers. [More Info](https://www.opentable.com/cuisine/best-italian-restaurants-downtown-chicago-il)

2. **Giordano's** - Known for serving authentic Chicago-style deep dish pizza with quality toppings, this restaurant also offers a selection of sandwiches, pastas, and buffalo wings in a clean, bright space. [More Info](https://www.tripadvisor.com/Restaurants-g35805-c26-zfn7778523-Chicago_Illinois.html)

3. **Monteverde** - Located in the West Loop, this Italian restaurant is known for its chef-owner, Sarah Grueneberg, and offers a delightful selection of pasta and pizza. [More Info](https://www.timeout.com/chicago/restaurants/best-italian-restaurants-in-chicago-find-pasta-pizza-and-more)

4. **Eater Chicago** - This restaurant offers comfort foods like tortellini alla panna and baked clams, making it a great spot for an early dinner for families with young children. [More Info](https://chicago.eater.com/maps/best-italian-restaurants-in-chicago)

### Sushi Restaurants
1. **SUSHI-SAN** - River North
   - Praised as one of the best Japanese restaurants in the US, offering a great dining experience with reasonable prices. It's a beloved sushi spot in Chicago featuring a diverse menu and a superb whiskey list. Many customers commend the attentive and friendly service. [More Info](https://www.opentable.com/cuisine/best-sushi-restaurants-downtown-chicago-il)

2. **Sushi Plus Rotary Sushi Bar** - Chinatown
   - Known for its rotating sushi bar and a variety of sushi options. It's a popular spot for sushi enthusiasts. [More Info](https://www.yelp.com/search?find_desc=sushi+downtown&find_loc=Chicago%2C+IL)

3. **Nobu Chicago**
   - Offers a modern dining experience with a diverse sushi menu and a stylish ambiance. [More Info](https://www.yelp.com/search?find_desc=sushi+downtown&find_loc=Chicago%2C+IL)

4. **Sunda Chicago**
   - A fusion of Asian flavors and modern cuisine, known for its creative sushi offerings and vibrant atmosphere. [More Info](https://chicago.eater.com/maps/best-sushi-restaurants-chicago)

5. **KAI ZAN**
   - A popular sushi restaurant known for its high-quality and creative sushi dishes. [More Info](https://www.yelp.com/search?find_desc=sushi+restaurants+downtown&find_loc=Chicago%2C+IL)

Feel free to explore these options and enjoy a delightful Italian and Sushi dining experience in downtown Chicago!

In [25]:
printmd(brain_agent_executor.invoke({"question": "chatgpt, tell me the formula in physics for momentum"}, config=config)["output"])

The formula for momentum in physics is given by:

\[ \text{Momentum} = \text{Mass} \times \text{Velocity} \]

Where:
- Momentum is the product of an object's mass and its velocity.
- Mass is the amount of matter in an object, usually measured in kilograms (kg).
- Velocity is the speed of an object in a particular direction, usually measured in meters per second (m/s).

This formula is fundamental in understanding the motion of objects and is a key concept in classical mechanics.

The formula for momentum in physics is given by:

\[ \text{Momentum} = \text{Mass} \times \text{Velocity} \]

Where:
- Momentum is the product of an object's mass and its velocity.
- Mass is the amount of matter in an object, usually measured in kilograms (kg).
- Velocity is the speed of an object in a particular direction, usually measured in meters per second (m/s).

This formula is fundamental in understanding the motion of objects and is a key concept in classical mechanics.

In [26]:
printmd(brain_agent_executor.invoke({"question": "docsearch, what can markov chains do?"}, config=config)["output"])

Agent Action: 
Invoking: `docsearch` with `markov chains`



Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'markov chains'}`



A Markov chain is a stochastic model that describes a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. Here are some insights from various sources related to Markov chains:

1. **Finite Markov Chains**:
   - A discrete Markov chain is completely described by its initial distribution and its transition probability matrix^1. The transition probability matrix Q is stochastic, i.e., all entries are non-negative, all column sums are equal to 1, and no row contains only zeros.
   - Markov chains and signal models are related by associating time/space marks to the states and the shift matrix to the transition probability matrix Q^1.
   - The application of Markov chains in signal models provides a rigorous mathematical connection^1.

2. **Particle Transport Prediction**

I've gathered some insights on what Markov chains can do based on various sources:

1. **Finite Markov Chains**: These are described by an initial distribution and a transition probability matrix. They are used in signal models and provide a rigorous mathematical connection.

2. **Particle Transport Prediction**: Markov chains are used in a combined computational fluid dynamics (CFD) and method for predicting transient particle transport in enclosed environments, providing faster-than-real-time information about particle transport.

3. **Fast Fluid Dynamics and Markov Chain Model**: A model has been proposed for predicting transient particle transport indoors, showing good agreement with experimental data and significant computational efficiency.

4. **Spatial Markov Chain Model for Virus Spread**: A spatial Markov chain model has been developed for the spread of viruses, representing a graph connecting nodes that represent humans and incorporating various lockdown scenarios.

5. **Genetic Fitness Optimization**: Markov chains are used in genetic computing for fitness optimization, and rapidly mixing Markov chains have been the basis of randomized approximation algorithms, combinatorial counting, statistical physics, and combinatorial optimization.

These applications demonstrate the versatility and significance of Markov chains in various domains, including physics, computational modeling, and optimization.

In [27]:
printmd(brain_agent_executor.invoke({"question": "@sqlsearch, How many people died of covid in Texas in 2020?"}, config=config)["output"])

Agent Action: 
Invoking: `sqlsearch` with `How many people died of covid in Texas in 2020?`



Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with ``



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`



Agent Action: 
Invoking: `sql_db_query` with `SELECT SUM(death) as total_deaths FROM covidtracking WHERE state='TX' AND date LIKE '2020%'`



The total number of people who died of COVID in Texas in 2020 is 2,841,253. 

Here's the query I used to obtain this result:
```sql
SELECT SUM(death) as total_deaths
FROM covidtracking
WHERE state='TX' AND date LIKE '2020%'
```In 2020, a total of 2,841,253 people died of COVID in Texas. This information is based on the data from the COVID tracking database. If you need further details or additional information, feel free to ask!

In 2020, a total of 2,841,253 people died of COVID in Texas. This information is based on the data from the COVID tracking database. If you need further details or additional information, feel free to ask!

In [28]:
printmd(brain_agent_executor.invoke({"question": "docsearch, I don't know how to say No to my kids, help me! What kind of boundaries should I set?"}, config=config)["output"])

Setting boundaries with your kids is an important aspect of parenting. Here are some key boundaries that you can consider setting to maintain a healthy and respectful relationship with your children:

### 1. Clear Communication
   - Clearly communicate your expectations and rules to your kids.
   - Use simple and age-appropriate language to explain the reasons behind the rules.

### 2. Consistency
   - Be consistent in enforcing the boundaries you set. Inconsistency can lead to confusion and frustration for children.

### 3. Respectful Behavior
   - Encourage respectful behavior towards others, including family members, peers, and adults.
   - Teach them to express their emotions and opinions in a respectful manner.

### 4. Screen Time and Technology
   - Set limits on screen time and establish rules for the use of electronic devices.
   - Encourage alternative activities such as reading, outdoor play, and creative hobbies.

### 5. Chores and Responsibilities
   - Assign age-appropriat

Setting boundaries with your kids is an important aspect of parenting. Here are some key boundaries that you can consider setting to maintain a healthy and respectful relationship with your children:

### 1. Clear Communication
   - Clearly communicate your expectations and rules to your kids.
   - Use simple and age-appropriate language to explain the reasons behind the rules.

### 2. Consistency
   - Be consistent in enforcing the boundaries you set. Inconsistency can lead to confusion and frustration for children.

### 3. Respectful Behavior
   - Encourage respectful behavior towards others, including family members, peers, and adults.
   - Teach them to express their emotions and opinions in a respectful manner.

### 4. Screen Time and Technology
   - Set limits on screen time and establish rules for the use of electronic devices.
   - Encourage alternative activities such as reading, outdoor play, and creative hobbies.

### 5. Chores and Responsibilities
   - Assign age-appropriate chores and responsibilities to teach accountability and teamwork.
   - Acknowledge and praise their efforts when they fulfill their responsibilities.

### 6. Personal Space and Privacy
   - Teach the importance of personal space and privacy.
   - Respect their boundaries and encourage open communication about personal matters.

### 7. Healthy Lifestyle
   - Set boundaries around healthy eating habits, physical activity, and bedtime routines.
   - Encourage a balanced lifestyle and model healthy behaviors.

### 8. Consequences and Discipline
   - Clearly outline the consequences of breaking rules and follow through with appropriate discipline.
   - Use positive reinforcement and constructive feedback to encourage positive behavior.

### 9. Mutual Respect
   - Foster an environment of mutual respect where children's opinions and feelings are valued.
   - Encourage open dialogue and active listening.

By setting these boundaries, you can create a nurturing and supportive environment for your children while teaching them valuable life skills and values. Remember that setting boundaries is a continuous process and requires patience, empathy, and understanding.

In [29]:
# This question although does not contain instructions for a tool, the brain agent decides what tool to use
printmd(brain_agent_executor.invoke({"question": "What's a good place to dine today in downtown Seoul?"}, config=config)["output"])

Agent Action: 
Invoking: `bing` with `best restaurants in downtown Seoul`



Tool: bing
Agent Action: 
Invoking: `Searcher` with `{'query': 'best restaurants in downtown Seoul'}`



Based on the information gathered, here are some of the best restaurants in downtown Seoul:

1. **Bar Cham**
   - This restaurant made the 2022 Asia’s 50 Best Bars list and offers various non-alcoholic cocktail options just as delicious as their alcoholic counterparts. 
   - [More Info](https://www.eater.com/maps/best-seoul-restaurants-38)

2. **Mingles Restaurant**
   - Cuisine: Korean (traditional and modern)
   - Price Range: $$$$
   - Nestled near the Han River in the Gangnam-Gu district, Mingles Restaurant is one of the most popular eateries in Seoul, known for impeccable service and delicious food.
   - [More Info](https://thekoreanguide.com/best-restaurants-seoul/)

3. **THE PLACE Dining**
   - Cuisine: Italian
   - This restaurant offers stunning views of the N Seoul Tower and is situated in the 2nd

Based on the information gathered, here are some of the best and top-rated restaurants in downtown Seoul:

### Best Restaurants in Downtown Seoul
1. **Bar Cham**
   - This restaurant made the 2022 Asia’s 50 Best Bars list and offers various non-alcoholic cocktail options just as delicious as their alcoholic counterparts. [More Info](https://www.eater.com/maps/best-seoul-restaurants-38)

2. **Mingles Restaurant**
   - Cuisine: Korean (traditional and modern)
   - Price Range: $$$$
   - Nestled near the Han River in the Gangnam-Gu district, Mingles Restaurant is one of the most popular eateries in Seoul, known for impeccable service and delicious food. [More Info](https://thekoreanguide.com/best-restaurants-seoul/)

3. **THE PLACE Dining**
   - Cuisine: Italian
   - This restaurant offers stunning views of the N Seoul Tower and is situated in the 2nd floor of N Seoul Tower Plaza, featuring a 360-degree view of Seoul with its window counter seating. [More Info](https://thesmartlocal.com/read/korea-scenic-restaurants/)

### Top-Rated Restaurants in Downtown Seoul
1. [Eater - The 38 Best Restaurants in Seoul, South Korea](https://www.eater.com/maps/best-seoul-restaurants-38)
   - This list features some of the best restaurants in Seoul, offering a variety of dining options.

2. [Tripadvisor - THE 10 BEST Restaurants in Seoul (Updated March 2024)](https://www.tripadvisor.com/Restaurants-g294197-Seoul.html)
   - Tripadvisor provides a list of the top-rated restaurants in Seoul based on customer reviews and ratings.

3. [Seoul MICHELIN Restaurants - The MICHELIN Guide South Korea](https://guide.michelin.com/en/kr/seoul-capital-area/kr-seoul/restaurants)
   - The Michelin Guide features the best restaurants in Seoul, including those with Michelin stars.

4. [TheKoreanGuide.com - 15 Best Restaurants in Seoul for Locals and Foreigners](https://thekoreanguide.com/best-restaurants-seoul/)
   - This guide highlights the best restaurants in Seoul, catering to both locals and foreigners, with a focus on Korean cuisine.

5. [Seoul Space - Top 10 Michelin Star Restaurants in Seoul](https://seoulspace.com/top-10-michelin-star-restaurants-in-seoul-seoul-space/)
   - This article lists the top Michelin-starred restaurants in Seoul, offering a premium dining experience.

You can explore these resources to find the perfect dining experience in downtown Seoul. Let me know if there's anything else I can assist you with!

In [30]:
# This question many times causes a parsing error, but we can still give the answer using the run_agent function
# which handles the parsing error exception
printmd(brain_agent_executor.invoke({"question": "@chatgpt, can you give me a javascript example of how to trim the spaces of a sentence?"}, config=config)["output"])

Agent Action: 
Invoking: `chatgpt` with `Sure! Here's a simple JavaScript example to trim the spaces of a sentence:

```javascript
let sentence = '   Hello, this is a sentence with spaces.   ';
let trimmedSentence = sentence.trim();
console.log(trimmedSentence);
````



Tool: chatgpt
Agent Action: 
Invoking: `sqlsearch` with `javascript trim spaces example`



Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with ``



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`



Agent Action: 
Invoking: `sql_db_query` with `SELECT TOP 10 state, TRIM(state) AS trimmed_state FROM covidtracking`



The SQL query used to obtain the trimmed state names is:
```sql
SELECT TOP 10 state, TRIM(state) AS trimmed_state FROM covidtracking
```

The result of the query shows the original state names and their corresponding trimmed state names:
| state | trimmed_state |
|-------|---------------|
| AK    | AK            |
| AL    | AL            |
| AR    | AR       

Here's a simple JavaScript example to trim the spaces of a sentence:

```javascript
let sentence = '   Hello, this is a sentence with spaces.   ';
let trimmedSentence = sentence.trim();
console.log(trimmedSentence);
// Output: "Hello, this is a sentence with spaces."
```

The `trim()` method removes leading and trailing spaces from a string, effectively trimming the spaces from both ends of the sentence. This ensures that the sentence is free from unnecessary spaces.

In [31]:
# This question should trigger our prompt safety instructions
printmd(brain_agent_executor.invoke({"question": "Tell me a funny joke about the president"}, config=config)["output"])

I'm sorry, I cannot comply with that request.

I'm sorry, I cannot comply with that request.

In [32]:
printmd(brain_agent_executor.invoke({"question": "Thank you!"}, config=config)["output"])

You're welcome! If you have any more questions or need further assistance, feel free to ask. Have a great day!

You're welcome! If you have any more questions or need further assistance, feel free to ask. Have a great day!

# Summary

Great!, We just built the GPT Smart Search Engine!
In this Notebook we created the brain, the decision making Agent that decides what Tool to use to answer the question from the user. This is what was necessary in order to have an smart chat bot.

We can have many tools to accomplish different tasks, including connecting to APIs, dealing with File Systems, and even using Humans as Tools. For more reference see [HERE](https://python.langchain.com/docs/integrations/tools/)

# NEXT
It is time now to use all the functions and prompts build so far and build a Web application.
The Next notebook will guide you on how to build:

1) A Bot API Backend
2) A Frontend UI with a Search and Webchat interfaces